In [2]:
import pandas as pd
import numpy as np
import glob
from nilearn import plotting
import matplotlib.pyplot as plt
from nilearn import image
from nilearn.glm.first_level import FirstLevelModel
from nilearn import plotting
from nilearn.glm import threshold_stats_img
from nilearn.glm.first_level import make_first_level_design_matrix
import nibabel as nib

In [3]:
# Load the data by entering the local path
root = ''
func_data_run_1 = glob.glob(root + 'sub-*/ses-*/func/*_run-1*desc-preproc_bold.nii.gz')
func_data_run_2 = glob.glob(root + 'sub-*/ses-*/func/*_run-2*desc-preproc_bold.nii.gz')
subj_T1 = glob.glob(root + 'sub-*/ses-*/anat/*space-*preproc_T1w.nii.gz') #pick a specific T1 for background image

# Define some params
nr_sub = 1
n_scans = 188 #total = 564 / 3 = 188 (e1, e2, e3)
tr = 1.63 #how much time it takes to acquire 1 volume
nr_cond = 7 #words, faces, numbers, places, objects, bodies, baseline
epoch_duration = 6.0 # in sec
nr_cont_gp = 16 


# Get the confounds file
conf_data_run_1 = glob.glob(root + 'sub-*/ses-*/func/*_run-1_desc-confounds_timeseries.tsv')
conf_data_run_2 = glob.glob(root + 'sub-*/ses-*/func/*_run-2_desc-confounds_timeseries.tsv')

# Create a list of dataframes with motion confounds and framewise disp
motion_conf_all=[]
for n in range(nr_sub):
    temp_df = pd.read_csv(conf_data_run_1[n], delimiter='\t')
    temp_df_2 = pd.read_csv(conf_data_run_2[n], delimiter='\t')
    sub_motion_params_1 = pd.DataFrame(temp_df, columns=['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z', 'framewise_displacement'])
    sub_motion_params_1['framewise_displacement'][0] = 0 # Change the first elem to 0 from Nan
    sub_motion_params_2 = pd.DataFrame(temp_df_2, columns=['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z', 'framewise_displacement'])
    sub_motion_params_2['framewise_displacement'][0] = 0 # Change the first elem to 0 from Nan
    motion_conf_all.append([sub_motion_params_1, sub_motion_params_2])


# Get the event onsets
onsets_dir = root + 'sub-control_task-floc_run-1_events.tsv'
events_allsub = []
for n in range(nr_sub):
    events_df = pd.read_csv(onsets_dir, sep='\t')
    events_allsub.append([events_df, events_df]) #nr_sub multiply events for all sub

#Concatenate run-1 and run-2 for each patient
func_data=[]
for n in range(nr_sub):
    func_data.append([image.concat_imgs(func_data_run_1[n], auto_resample=False), image.concat_imgs(func_data_run_2[n], auto_resample=False)])

# Smoothing
func_data_sm = [] 
for n in range(nr_sub):
    func_data_sm.append(image.smooth_img(func_data[n], fwhm=4))

frame_times = np.arange(n_scans) * tr
design_matrices = []
for n in range(nr_sub):
    design_matrices.append([make_first_level_design_matrix(frame_times, events=events_df, hrf_model='spm'), make_first_level_design_matrix(frame_times, events=events_df, hrf_model='spm')])

# contrast matrix
contrast_matrix = np.eye(design_matrices[0][0].shape[1]) #double because its nested
contrasts = dict([(column, contrast_matrix[i])
                  for i, column in enumerate(design_matrices[0][0].columns)])
design_matrix = design_matrices[0][0].reset_index()

fmri_glm_single_subj = FirstLevelModel(tr, noise_model='ar1', standardize=False, hrf_model=hrf_model)
fmri_glm_single_subj = fmri_glm_single_subj.fit(func_data_sm[0], events= events_allsub[0])

In [ ]:
def plot_interactive_floc(glm, subj_id, root, alpha):

    #need to set specific contrasts?? and generate all html versions?
    contrast = (contrasts['Faces'], contrasts['Faces'])
    test_contrast = {"faces-oth": contrast}

    subj_T1 = glob.glob(root + f'sub-{subj_id}/ses-*/anat/*space-*preproc_T1w.nii.gz') #pick a specific T1 for background image
    subj_T1_img = image.load_img(subj_T1)

    for contrast_id, contrast_val in test_contrast.items():
            z_map = glm.compute_contrast(contrast_val, output_type='z_score')
            z_img_thresh, thresholded = threshold_stats_img(z_map, alpha=alpha, height_control="fpr", cluster_threshold=20, two_sided = False)
    view = plotting.view_img(
        z_img_thresh,
        colorbar=True,
        bg_img=subj_T1_img,
        cut_coords=[-45, -57, -12])
   
    view.save_as_html(f'{subj_id}_alpha{alpha}_interactive_floc.html') #add path to where we want to save floc.html here

In [ ]:
glm = fmri_glm_single_subj
subj_id = '1234'
root = ''
alpha_list = [0.001, 0.01, 0.05]

for alpha in alpha_list:
    plot_interactive_floc(glm, subj_id, root, alpha)